Importing the useful libraries

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import pbs
#from pickle_vocab import *
#from cooc import *

## Loading the data

In [5]:
def load_test_data(data_path):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    
    f = open(data_path, encoding='utf-8')
    x = f.readlines()
    tweets = []
    ids = []
    for str in x :
        id, tweet = str.split(',',1)
        tweets.append(tweet)
        ids.append(id)

    x = pd.DataFrame(tweets,ids,columns=['Tweets'])
    
    return x

test = load_test_data('Datasets/test_data.txt')
#test.head()

In [6]:
def load_train_data(data_path):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    
    f = open(data_path, encoding="utf-8")
    x = f.readlines()
    x = pd.DataFrame(x,columns=['Tweets'])
    
    return x

In [7]:
pos = load_train_data('Datasets/train_pos.txt')
pos_full = load_train_data('Datasets/train_pos_full.txt')
#pos.head()

In [8]:
neg = load_train_data('Datasets/train_neg.txt')
neg_full = load_train_data('Datasets/train_neg_full.txt')
#neg.head()

In [9]:
pos["y"]=1
pos_full["y"]=1
#pos.head()

In [10]:
neg["y"]=-1
neg_full["y"]=-1
#neg.head()

In [11]:
train = pd.concat([pos,neg])
train.reset_index(drop=True,inplace=True)
train_full = pd.concat([pos_full,neg_full])
train_full.reset_index(drop=True,inplace=True)
#train.head()

In [13]:
del train
del pos
del pos_full
del neg
del neg_full

NameError: name 'pos' is not defined

In [214]:
print("following rows get NaN weights")
train.loc[[29049, 52357, 56151, 68379, 69973, 114433, 133011, 175789, 190542]]

following rows get NaN weights


,Tweets,y
29049,heyheyheyhey\n,1
52357,#ireland #ireland #ireland\n,1
56151,gilipollas #everywhere\n,1
68379,#absolutely beastingit\n,1
69973,#haveagooddaytomorrow #goodnightworld\n,1
114433,#killingmesoftly\n,-1
133011,catss everywheree\n,-1
175789,straks nederland toets\n,-1
190542,ziekjes thuis\n,-1


In [215]:
test.loc[['4590']]

,Tweets
4590,#thatsjustmethough\n


In [33]:
with open('vocab.pkl','rb') as f:
    vocab = pickle.load(f)

vocab, vocab_full

({'<user>': 0,
  '!': 1,
  'i': 2,
  'the': 3,
  '.': 4,
  ',': 5,
  'to': 6,
  'you': 7,
  '(': 8,
  '<url>': 9,
  'a': 10,
  '...': 11,
  'and': 12,
  'my': 13,
  'me': 14,
  'of': 15,
  '?': 16,
  'is': 17,
  'for': 18,
  'in': 19,
  'it': 20,
  '"': 21,
  'this': 22,
  'so': 23,
  '-': 24,
  'with': 25,
  'on': 26,
  'that': 27,
  ')': 28,
  'be': 29,
  "i'm": 30,
  'have': 31,
  ':': 32,
  'but': 33,
  'just': 34,
  'rt': 35,
  'love': 36,
  'your': 37,
  'all': 38,
  'not': 39,
  'was': 40,
  'at': 41,
  'are': 42,
  '..': 43,
  'like': 44,
  '/': 45,
  'get': 46,
  'up': 47,
  'frame': 48,
  '&': 49,
  'lol': 50,
  'know': 51,
  'good': 52,
  'do': 53,
  'u': 54,
  'now': 55,
  'when': 56,
  'one': 57,
  'if': 58,
  'we': 59,
  'follow': 60,
  'no': 61,
  'can': 62,
  'what': 63,
  'go': 64,
  "don't": 65,
  'out': 66,
  'x': 67,
  'will': 68,
  'day': 69,
  "'": 70,
  'please': 71,
  'from': 72,
  'see': 73,
  'too': 74,
  'want': 75,
  'there': 76,
  'back': 77,
  "it's": 78,


In [15]:
with open('vocab_full.pkl','rb') as f:
    vocab_full = pickle.load(f)

In [217]:
max(vocab.values())

21160

In [4]:
#!/usr/bin/env python3
import pickle


def pickle_vocab_full():
    vocab = dict()
    with open('vocab_cut_full.txt') as f:
        for idx, line in enumerate(f):
            vocab[line.strip()] = idx

    with open('vocab_full.pkl', 'wb') as f:
        pickle.dump(vocab, f, pickle.HIGHEST_PROTOCOL)
    
pickle_vocab_full()

In [5]:
#!/usr/bin/env python3
from scipy.sparse import *
import numpy as np
import pickle


def cooc_full():
    with open('vocab_full.pkl', 'rb') as f:
        vocab = pickle.load(f)
    vocab_size = len(vocab)

    data, row, col = [], [], []
    counter = 1
    for fn in ['Datasets/train_pos_full.txt', 'Datasets/train_neg_full.txt']:
        with open(fn) as f:
            for line in f:
                tokens = [vocab.get(t, -1) for t in line.strip().split()]
                tokens = [t for t in tokens if t >= 0]
                for t in tokens:
                    for t2 in tokens:
                        data.append(1)
                        row.append(t)
                        col.append(t2)

                if counter % 10000 == 0:
                    print(counter)
                counter += 1
    cooc = coo_matrix((data, (row, col)))
    print("summing duplicates (this can take a while)")
    cooc.sum_duplicates()
    with open('cooc_full.pkl', 'wb') as f:
        pickle.dump(cooc, f, pickle.HIGHEST_PROTOCOL)
        
        
cooc_full()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

MemoryError: 

In [218]:
with open('cooc.pkl','rb') as f:
    cooc = pickle.load(f)
cooc

<21160x21160 sparse matrix of type '<class 'numpy.int32'>'
	with 3184638 stored elements in COOrdinate format>

In [219]:
cooc.data

array([273434,  97020,  57576, ...,      2,      2,      8])

In [220]:
cooc.row

array([    0,     1,     2, ..., 12840, 14651, 21159])

In [221]:
cooc.col

array([    0,     0,     0, ..., 21159, 21159, 21159])

In [222]:
print(cooc)

  (0, 0)	273434
  (1, 0)	97020
  (2, 0)	57576
  (3, 0)	38886
  (4, 0)	47366
  (5, 0)	41118
  (6, 0)	41428
  (7, 0)	58492
  (8, 0)	4744
  (9, 0)	10364
  (10, 0)	31758
  (11, 0)	12300
  (12, 0)	27856
  (13, 0)	21426
  (14, 0)	22728
  (15, 0)	14316
  (16, 0)	29786
  (17, 0)	15902
  (18, 0)	20674
  (19, 0)	14742
  (20, 0)	21172
  (21, 0)	19938
  (22, 0)	8938
  (23, 0)	12654
  (24, 0)	6002
  :	:
  (208, 21159)	4
  (239, 21159)	2
  (250, 21159)	2
  (273, 21159)	2
  (314, 21159)	6
  (349, 21159)	2
  (444, 21159)	2
  (676, 21159)	2
  (693, 21159)	2
  (883, 21159)	2
  (1048, 21159)	2
  (1050, 21159)	2
  (1388, 21159)	2
  (1415, 21159)	2
  (2291, 21159)	2
  (2915, 21159)	2
  (3359, 21159)	2
  (6055, 21159)	2
  (7507, 21159)	2
  (7875, 21159)	2
  (7879, 21159)	2
  (9534, 21159)	2
  (12840, 21159)	2
  (14651, 21159)	2
  (21159, 21159)	8


In [223]:
cooc.toarray()

array([[273434,  97020,  57576, ...,      0,     10,      4],
       [ 97020, 255078,  47124, ...,      0,      2,      0],
       [ 57576,  47124, 110506, ...,      0,      0,      4],
       ...,
       [     0,      0,      0, ...,      0,      0,      0],
       [    10,      2,      0, ...,      0,     10,      0],
       [     4,      0,      4, ...,      0,      0,      8]])

In [224]:
cooc.toarray().shape

(21160, 21160)

In [16]:
#we = np.load("embeddings.npy")
we_full = np.load("embeddings_full.npy")
we_full

array([[-0.00790058, -0.00630557,  0.00753521, ..., -0.0283808 ,
         0.01144536, -0.00558127],
       [ 0.0029428 ,  0.01254714, -0.01901694, ..., -0.02141594,
         0.02366955,  0.02600938],
       [-0.00650199, -0.00820677, -0.00627628, ..., -0.02379747,
         0.00326442,  0.00680607],
       ...,
       [ 0.49645731, -1.61725388, -0.0079772 , ..., -0.25880692,
        -0.34577931,  0.70514542],
       [ 0.52614538,  0.36968974, -0.77102577, ..., -1.64859207,
         0.39028181, -2.50558299],
       [-0.32842242,  1.13262929, -1.06326081, ..., -0.60556867,
         0.75839111,  1.09999427]])

In [226]:
we.shape

(21160, 20)

In [9]:
#!/usr/bin/env python3
from scipy.sparse import *
import numpy as np
import pickle
import random



print("loading cooccurrence matrix")
with open('cooc_full.pkl', 'rb') as f:
    cooc = pickle.load(f)
print("{} nonzero entries".format(cooc.nnz))

nmax = 100
print("using nmax =", nmax, ", cooc.max() =", cooc.max())

print("initializing embeddings")
embedding_dim = 20
xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
ys = np.random.normal(size=(cooc.shape[1], embedding_dim))

eta = 0.001
alpha = 3 / 4

epochs = 10

for epoch in range(epochs):
    print("epoch {}".format(epoch))
    for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
        logn = np.log(n)
        fn = min(1.0, (n / nmax) ** alpha)
        x, y = xs[ix, :], ys[jy, :]
        scale = 2 * eta * fn * (logn - np.dot(x, y))
        xs[ix, :] += scale * y
        ys[jy, :] += scale * x
np.save('embeddings_full', xs)

loading cooccurrence matrix
46596034 nonzero entries
using nmax = 100 , cooc.max() = 2599902
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [17]:
def representation(tweet, we, vocab):
    acc = np.array(0)
    n_ignored_word = 0
    for word in tweet.split():
        if word not in vocab.keys():
            n_ignored_word += 1
        else:
            try:
                acc = np.add(acc,we[vocab[word]])
            except: 
                print("problem with " + word) #last word from vocab is missing in cooc
                n_ignored_word += 1
    n = len(tweet.split()) - n_ignored_word
    acc = acc/n
    return(acc)

In [34]:
train["w"] = train["Tweets"].apply(lambda x: representation(x, we, vocab))

problem with #10factsaboutme
problem with #10factsaboutme
problem with #10factsaboutme
problem with #10factsaboutme
problem with #10factsaboutme


In [18]:
train_full["w"] = train_full["Tweets"].apply(lambda x: representation(x, we_full, vocab_full))

In [23]:
col = ["w" + str(k) for k in range(20)]
col

['w0',
 'w1',
 'w2',
 'w3',
 'w4',
 'w5',
 'w6',
 'w7',
 'w8',
 'w9',
 'w10',
 'w11',
 'w12',
 'w13',
 'w14',
 'w15',
 'w16',
 'w17',
 'w18',
 'w19']

In [22]:
train_full.drop("Tweets",axis=1,inplace=True)

KeyError: "['Tweets'] not found in axis"

In [24]:
#train.drop("Tweets",axis=1,inplace=True)
#train[col] = train["w"].apply(pd.Series)
#train.drop("w",axis=1,inplace=True)



train_full[col] = train_full["w"].apply(pd.Series)
train_full.drop("w",axis=1,inplace=True)

In [25]:
with open('train_full.pkl', 'wb') as f:
        pickle.dump(train_full, f, pickle.HIGHEST_PROTOCOL)

In [26]:
from IPython.display import display

display(train_full.head())

,y,w0,w1,w2,w3,w4,w5,w6,w7,w8,...,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19
0,1,-0.296790,-0.181150,-0.058083,0.322222,-0.542621,-0.024000,0.140801,-0.151043,0.080479,...,0.065230,0.089998,-0.074064,0.487775,-0.113918,0.182005,0.249724,-0.118905,0.098660,0.156168
1,1,-0.202436,-0.280010,-0.107246,0.239919,-0.509784,-0.033202,0.081003,-0.129831,0.085327,...,0.118092,0.126240,0.022105,0.426787,-0.094366,0.178490,0.300231,-0.039059,0.122455,0.086746
2,1,-0.220637,-0.083819,0.100908,0.222788,-0.278018,-0.098640,0.052325,-0.099279,-0.011704,...,0.044298,0.069571,-0.029328,0.195117,0.058874,0.114208,0.135762,-0.115157,0.062747,0.111751
3,1,-0.268295,-0.193417,-0.071036,0.369684,-0.534334,0.037926,0.223013,-0.237248,-0.027922,...,0.064163,0.087668,-0.104771,0.604485,-0.082731,0.160737,0.253678,-0.089962,0.072788,0.119213
4,1,-0.106054,-0.253317,0.099771,0.030015,-0.266051,0.050594,0.118562,-0.103226,0.025749,...,0.090308,0.148502,-0.104850,0.293243,-0.184430,0.301689,0.187145,-0.092530,-0.059525,0.197912


In [232]:
test.head()

,Tweets
1,sea doo pro sea scooter ( sports with the port...
2,<user> shucks well i work all week so now i ca...
3,i cant stay away from bug thats my baby\n
4,<user> no ma'am ! ! ! lol im perfectly fine an...
5,"whenever i fall asleep watching the tv , i alw..."


In [27]:
test["w"] = test["Tweets"].apply(lambda x: representation(x, we_full, vocab_full))
test[col] = test["w"].apply(pd.Series)
test.drop("w",axis=1,inplace=True)
test.drop("Tweets",axis=1,inplace=True)

In [234]:
test

,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19
1,0.259533,0.059221,-0.450866,-0.689856,0.480041,-0.267784,0.051785,0.359576,0.494638,0.120045,-0.185360,0.557172,0.507819,0.107620,0.542991,-0.045309,0.342527,-0.196790,-0.439141,-0.308582
2,0.150485,-0.073187,-0.203847,-0.686830,0.483774,-0.184477,0.248559,0.150328,0.346024,-0.007975,-0.261741,0.296996,0.547765,-0.002013,0.708832,0.237233,0.631884,-0.254344,-0.244305,-0.268326
3,0.109122,-0.055159,-0.075084,-0.602511,0.323006,-0.402221,0.205848,0.491140,0.413059,-0.107106,-0.222141,0.342126,0.476687,0.256765,0.619973,0.047600,0.717036,-0.269757,-0.305203,-0.229819
4,0.200103,0.153898,-0.171113,-0.550022,0.206411,-0.279831,0.057420,0.361296,0.258558,-0.019996,-0.122725,0.322543,0.276396,0.134913,0.817255,-0.026374,0.644372,-0.504629,-0.206761,-0.277321
5,0.271728,-0.070988,-0.006069,-0.511087,0.414814,-0.379588,0.132693,0.283727,0.483064,0.062769,-0.295117,0.355119,0.554627,0.214441,0.567460,0.126346,0.606131,-0.426146,-0.315099,-0.230338
6,0.177140,-0.064049,-0.235508,-0.671921,0.468047,-0.351670,0.123075,0.417831,0.360304,-0.032732,-0.116584,0.438575,0.328540,0.272162,0.622874,0.154174,0.768459,-0.156917,-0.233959,-0.193053
7,0.050540,0.023630,-0.318031,-0.700890,0.656585,-0.121047,0.151539,0.208249,0.376164,-0.042006,-0.146891,0.424010,0.646687,0.260476,0.660293,-0.085088,0.691258,-0.264102,-0.326783,-0.194410
8,0.079734,-0.147615,-0.081583,-0.771016,0.457424,-0.368448,0.115835,0.293994,0.370616,-0.259839,-0.174606,0.229767,0.545067,0.234551,0.754708,-0.037113,0.669069,-0.298692,-0.250571,-0.286325
9,0.069434,-0.049755,-0.068777,-0.770877,0.539732,-0.381339,0.183634,0.351502,0.487904,-0.127915,-0.218412,0.373151,0.487160,0.101285,0.847474,0.132254,0.741720,-0.297788,-0.352823,-0.340416
10,0.088795,-0.036685,-0.089279,-0.693793,0.477863,-0.204369,0.208715,0.417147,0.443159,-0.089004,-0.273531,0.289795,0.490565,0.098937,0.548331,0.185409,0.798275,-0.205321,-0.206249,-0.303574


In [28]:
train_full[train_full.isnull().any(axis=1)].index

Int64Index([  29049,   56151,   68379,  114187,  152738,  214353,  224611,
             240835,  248364,  262903,  273027,  395411,  423819,  429381,
             445207,  522076,  541049,  601783,  655092,  672119,  688454,
             772457,  788210,  800748,  805714,  816453,  900218,  909214,
             920188,  994920, 1023925, 1026478, 1033297, 1039583, 1054132,
            1072735, 1105838, 1116982, 1116983, 1140450, 1147351, 1199854,
            1233454, 1264433, 1283011, 1366486, 1440168, 1553488, 1562646,
            1594451, 1614181, 1621006, 1646450, 1660651, 1716033, 1757957,
            1782742, 1809894, 1827675, 1840472, 1869113, 1945025, 2013870,
            2022990, 2071721, 2082029, 2089452, 2100361, 2130263, 2140897,
            2151061, 2167952, 2177220, 2303468, 2349139, 2386666, 2403651,
            2442445, 2467230],
           dtype='int64')

In [29]:
train_full.dropna(inplace=True)

In [30]:
test[test.isnull().any(axis=1)].index

Index(['4590'], dtype='object')

In [31]:
test.dropna(inplace=True)

In [32]:
X = train_full.drop("y", axis=1)
y = train_full["y"]

In [33]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier()
clf.fit(X, y) 
test["p"]=clf.predict(test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


You should first create a folder named **Submissions**

In [36]:
test["p"].to_csv("Submissions/submission_full.csv",header=False)

## Data Analysis

## Word enbeddings matrix

### Generation

### Visualization

## Building the text classifier

### Training texts features

### Linear classifiers

### Predictions

## Output creation and visualization

### Creating the submission file

### Visualization of the results